# SQL Database - Connect using Key Vault

In the previous lession ([Key Vault-backed Secret Scopes]($./08-Key-Vault-backed-secret-scopes)), you created a Key Vault-backed secret scope for Azure Databricks, and securely stored your Azure SQL username and password within.

In this lesson, you will use the Azure SQL Database secrets that are securely stored within the Key Vault-backed secret scope to connect to your Azure SQL Database instance, using the JDBC drivers that come with Databricks Runtime version 3.4 and above. Next, you will use the DataFrame API to execute SQL queries and control the parallelism of reads through the JDBC interface.

You will be using a new data set, as the customer has decided to make their e-commerce SQL data available for querying and processing within Azure Databricks. You will prove that you can successfully read data from and write data to the database using the JDBC driver.

## Access Key Vault secrets and configure JDBC connection

In the previous lesson, you created two secrets in Key Vault for your Azure SQL Database instance: **sql-username** and **sql-password**. You should have also made note of the Azure SQL Server host name and Database name. If you do not have this information, take a moment to retrieve those details from the Azure portal.

In [3]:
%scala
val jdbcUsername = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-username")
val jdbcPassword = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-password")

jdbcUsername: String = [REDACTED]
jdbcPassword: String = [REDACTED]

Notice that the values of `jdbcUsername` and `jdbcPassword` when printed out are `[REDACTED]`. This is to prevent your secrets from being exposed.

The next step is to ensure the JDBC driver is available.

In [5]:
%scala
Class.forName("com.microsoft.sqlserver.jdbc.SQLServerDriver")

res0: Class[_] = class com.microsoft.sqlserver.jdbc.SQLServerDriver

Now create a connection to the Azure SQL Database using the username and password from Key Vault, as well as the SQL Server host name and database name.

> The **host name** value will be in the following format: `<YOUR_SERVER_NAME>.databrickssqlserver.database.windows.net`.

In [7]:
# Create input widgets to store the host name and database values. This will allow us to access those same values from cells that use different languages.
# Execute this cell to display the widgets on top of the page, then fill the information before continuing to the next cell.
dbutils.widgets.text("hostName", "", "databrickssqlserver-dp-100.database.windows.net")
dbutils.widgets.text("database", "", "Databricks-SQL")

In [8]:
%scala
val jdbcHostname = dbutils.widgets.get("hostName")
val jdbcPort = 1433
val jdbcDatabase = dbutils.widgets.get("database")
val jdbcHostname1 = "databrickssqlserver-dp-100.database.windows.net"

// Create the JDBC URL without passing in the user and password parameters.
val jdbcUrl = s"jdbc:sqlserver://${jdbcHostname1}:${jdbcPort};database=${jdbcDatabase}"
print(jdbcUrl)
// Create a Properties() object to hold the parameters.
import java.util.Properties
val connectionProperties = new Properties()

connectionProperties.put("user", s"${jdbcUsername}")
connectionProperties.put("password", s"${jdbcPassword}")

jdbc:sqlserver://databrickssqlserver-dp-100.database.windows.net:1433;database=Databricks-SQLjdbcHostname: String = databrickssqlserver.database.windows.net
jdbcPort: Int = 1433
jdbcDatabase: String = Databricks-SQL
jdbcHostname1: String = databrickssqlserver-dp-100.database.windows.net
jdbcUrl: String = jdbc:sqlserver://databrickssqlserver-dp-100.database.windows.net:1433;database=Databricks-SQL
import java.util.Properties
connectionProperties: java.util.Properties = {user=[REDACTED], password=[REDACTED]}
res11: Object = null

Make sure you can connect to the Azure SQL Database, using the JDBC driver.

In [10]:
%scala
val driverClass = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
connectionProperties.setProperty("Driver", driverClass)

driverClass: String = com.microsoft.sqlserver.jdbc.SQLServerDriver
res12: Object = null

If there are no errors, then the connection was successful.

## Read data from JDBC

To start, we'll use a single JDBC connection to pull an Azure SQL Database table into the Spark environment. Your queries will be executed through the DataFrame API, providing a consistent experience whether you are querying flat files, Databricks tables, Azure Cosmos DB, SQL Server, or any other of the number of data sources you can access from Databricks.

In [13]:
%scala
val product_table = spark.read.jdbc(jdbcUrl, "SalesLT.Product", connectionProperties)

product_table: org.apache.spark.sql.DataFrame = [ProductID: int, Name: string ... 15 more fields]

Spark automatically reads the schema from the database table and maps its types back to Spark SQL types. You can see the schema using the `printSchema` command:

In [15]:
%scala
product_table.printSchema

root
-- ProductID: integer (nullable = true)
-- Name: string (nullable = true)
-- ProductNumber: string (nullable = true)
-- Color: string (nullable = true)
-- StandardCost: decimal(19,4) (nullable = true)
-- ListPrice: decimal(19,4) (nullable = true)
-- Size: string (nullable = true)
-- Weight: decimal(8,2) (nullable = true)
-- ProductCategoryID: integer (nullable = true)
-- ProductModelID: integer (nullable = true)
-- SellStartDate: timestamp (nullable = true)
-- SellEndDate: timestamp (nullable = true)
-- DiscontinuedDate: timestamp (nullable = true)
-- ThumbNailPhoto: binary (nullable = true)
-- ThumbnailPhotoFileName: string (nullable = true)
-- rowguid: string (nullable = true)
-- ModifiedDate: timestamp (nullable = true)

Now that we have our DataFrame, we can run queries against the JDBC table. Below, we are calculating the average list price by product category.

In [17]:
%scala
display(product_table.select("ProductCategoryID", "ListPrice")
        .groupBy("ProductCategoryID")
        .avg("ListPrice"))

ProductCategoryID,avg(ListPrice)
31,159.0
34,21.98
28,74.99
27,9.245
26,64.27571429
12,278.99
22,89.99
13,106.475
16,678.25357143
6,1597.45


It is possible to save the DataFrame out to a temporary view. This opens opportunities, such as querying using the SQL syntax, as you will see a few cells below.

In [19]:
%scala
product_table.createOrReplaceTempView("Product")
display(spark.sql("SELECT * FROM Product"))

ProductID,Name,ProductNumber,Color,StandardCost,ListPrice,Size,Weight,ProductCategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,ThumbNailPhoto,ThumbnailPhotoFileName,rowguid,ModifiedDate
680,"HL Road Frame - Black, 58",FR-R92B-58,Black,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMwAAZgAAmQAAzAAA/wAzAAAzMwAzZgAzmQAzzAAz/wBmAABmMwBmZgBmmQBmzABm/wCZAACZMwCZZgCZmQCZzACZ/wDMAADMMwDMZgDMmQDMzADM/wD/AAD/MwD/ZgD/mQD/zAD//zMAADMAMzMAZjMAmTMAzDMA/zMzADMzMzMzZjMzmTMzzDMz/zNmADNmMzNmZjNmmTNmzDNm/zOZADOZMzOZZjOZmTOZzDOZ/zPMADPMMzPMZjPMmTPMzDPM/zP/ADP/MzP/ZjP/mTP/zDP//2YAAGYAM2YAZmYAmWYAzGYA/2YzAGYzM2YzZmYzmWYzzGYz/2ZmAGZmM2ZmZmZmmWZmzGZm/2aZAGaZM2aZZmaZmWaZzGaZ/2bMAGbMM2bMZmbMmWbMzGbM/2b/AGb/M2b/Zmb/mWb/zGb//5kAAJkAM5kAZpkAmZkAzJkA/5kzAJkzM5kzZpkzmZkzzJkz/5lmAJlmM5lmZplmmZlmzJlm/5mZAJmZM5mZZpmZmZmZzJmZ/5nMAJnMM5nMZpnMmZnMzJnM/5n/AJn/M5n/Zpn/mZn/zJn//8wAAMwAM8wAZswAmcwAzMwA/8wzAMwzM8wzZswzmcwzzMwz/8xmAMxmM8xmZsxmmcxmzMxm/8yZAMyZM8yZZsyZmcyZzMyZ/8zMAMzMM8zMZszMmczMzMzM/8z/AMz/M8z/Zsz/mcz/zMz///8AAP8AM/8AZv8Amf8AzP8A//8zAP8zM/8zZv8zmf8zzP8z//9mAP9mM/9mZv9mmf9mzP9m//+ZAP+ZM/+ZZv+Zmf+ZzP+Z///MAP/MM//MZv/Mmf/MzP/M////AP//M///Zv//mf//zP///yH5BAEAABAALAAAAABQADEAAAj/AP8JHEiwoMGDCBMqXMiwocOHECNKnEixosWLGDNq3Mixo8ePIEOKHEmypMmTKFOqXJkRBYqBLhfGZPnQ5ct/MxPmpMnQpsCZNm/CfBnTZ86gQ3HeRMoRadGlQpUqJfoUZ9KnVH9GxVhUKtCoVaWKnZrVK9SmVMPuVHvWrFisPjd+LbuW7tmvb8t6nJuXIFutfbH2lSt07ta/eeOy3clTYuGtjS8yjUy5suXLmDHHdRjWIGPGIjdDBA3YL2SQVY+mvQsVL16yqLOqfuyWtlHZbTv+nY176G67H38DTs068GrSkoMSN+62+fKQqrW2Xe6aem7CSaf6fq7ceevTmcOLEh9Pvrz58+jTq1/Pvr379+8DAgA7,no_image_available_small.gif,43DD68D6-14A4-461F-9069-55309D90EA7E,2008-03-11T10:01:36.827+0000
706,"HL Road Frame - Red, 58",FR-R92R-58,Red,1059.31,1431.5,58,1016.04,18,6,2002-06-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMwAAZgAAmQAAzAAA/wAzAAAzMwAzZgAzmQAzzAAz/wBmAABmMwBmZgBmmQBmzABm/wCZAACZMwCZZgCZmQCZzACZ/wDMAADMMwDMZgDMmQDMzADM/wD/AAD/MwD/ZgD/mQD/zAD//zMAADMAMzMAZjMAmTMAzDMA/zMzADMzMzMzZjMzmTMzzDMz/zNmADNmMzNmZjNmmTNmzDNm/zOZADOZMzOZZjOZmTOZzDOZ/zPMADPMMzPMZjPMmTPMzDPM/zP/ADP/MzP/ZjP/mTP/zDP//2YAAGYAM2YAZmYAmWYAzGYA/2YzAGYzM2YzZmYzmWYzzGYz/2ZmAGZmM2ZmZmZmmWZmzGZm/2aZAGaZM2aZZmaZmWaZzGaZ/2bMAGbMM2bMZmbMmWbMzGbM/2b/AGb/M2b/Zmb/mWb/zGb//5kAAJkAM5kAZpkAmZkAzJkA/5kzAJkzM5kzZpkzmZkzzJkz/5lmAJlmM5lmZplmmZlmzJlm/5mZAJmZM5mZZpmZmZmZzJmZ/5nMAJnMM5nMZpnMmZnMzJnM/5n/AJn/M5n/Zpn/mZn/zJn//8wAAMwAM8wAZswAmcwAzMwA/8wzAMwzM8wzZswzmcwzzMwz/8xmAMxmM8xmZsxmmcxmzMxm/8yZAMyZM8yZZsyZmcyZzMyZ/8zMAMzMM8zMZszMmczMzMzM/8z/AMz/M8z/Zsz/mcz/zMz///8AAP8AM/8AZv8Amf8AzP8A//8zAP8zM/8zZv8zmf8zzP8z//9mAP9mM/9mZv9mmf9mzP9m//+ZAP+ZM/+ZZv+Zmf+ZzP+Z///MAP/MM//MZv/Mmf/MzP/M////AP//M///Zv//mf//zP///yH5BAEAABAALAAAAABQADEAAAj/AP8JHEiwoMGDCBMqXMiwocOHECNKnEixosWLGDNq3Mixo8ePIEOKHEmypMmTKFOqXJkRBYqBLhfGZPnQ5ct/MxPmpMnQpsCZNm/CfBnTZ86gQ3HeRMoRadGlQpUqJfoUZ9KnVH9GxVhUKtCoVaWKnZrVK9SmVMPuVHvWrFisPjd+LbuW7tmvb8t6nJuXIFutfbH2lSt07ta/eeOy3clTYuGtjS8yjUy5suXLmDHHdRjWIGPGIjdDBA3YL2SQVY+mvQsVL16yqLOqfuyWtlHZbTv+nY176G67H38DTs068GrSkoMSN+62+fKQqrW2Xe6aem7CSaf6fq7ceevTmcOLEh9Pvrz58+jTq1/Pvr379+8DAgA7,no_image_available_small.gif,9540FF17-2712-4C90-A3D1-8CE5568B2462,2008-03-11T10:01:36.827+0000
707,"Sport-100 Helmet, Red",HL-U509-R,Red,13.0863,34.99,null,null,35,33,2005-07-01T00:00:00.000+0000,null,null,R0lGODlhUAAxAPcAAAAAAIAAAACAAICAAAAAgIAAgACAgICAgMDAwP8AAAD/AP//AAAA//8A/wD//////wAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMwAAZgAAmQAAzAAA/wAzAAAzMwAzZgAzmQAzzAAz/wBmAABmMwBmZgBmmQBmzABm/wCZAACZMwCZZgCZmQCZzACZ/wDMAADMMwDMZgDMmQDMzADM/wD/AAD/MwD/ZgD/mQD/zAD//zMAADMAMzMAZjMAmTMAzDMA/zMzADMzMzMzZjMzmTMzzDMz/zNmADNmMzNmZjNmmT

Use the `%sql` magic to change the language of a cell to SQL. In the cell below, execute a SQL query that performs the same average `ListPrice` and product category grouping that was done using the DataFrame earlier.

In [21]:
%sql
select ProductCategoryID, AVG(ListPrice) from Product
Group By ProductCategoryID

ProductCategoryID,avg(ListPrice)
31,159.0
34,21.98
28,74.99
27,9.245
26,64.27571429
12,278.99
22,89.99
13,106.475
16,678.25357143
6,1597.45


You can also use the JDBC driver using Python. The cell below establishes a JDBC connection to the Azure SQL Database just as was done earlier using Scala.

Notice that you must define new parameters for the host name, database, username, and password. This is because those variables were defined using a different language earlier, which are inaccessible by a different language engine.

In [23]:
jdbcHostname = dbutils.widgets.get("hostName")
jdbcHostname = "databrickssqlserver-dp-100.database.windows.net"
jdbcDatabase = dbutils.widgets.get("database")
jdbcPort = 1433
print(jdbcHostname)
print(jdbcDatabase)

# Retrieve the database username and password from Key Vault
jdbcUsername = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-username")
jdbcPassword = dbutils.secrets.get(scope = "key-vault-secrets", key = "sql-password")
print(jdbcUsername)
print(jdbcPassword)
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

databrickssqlserver-dp-100.database.windows.net
Databricks-SQL
[REDACTED]
[REDACTED]

Now that the JDBC connection has been created in Python, let's query the JDBC connections across multiple workers. You do this by setting the `numPartitions` property. This determines how many connections used to push data through the JDBC API.

In [25]:
df = spark.read.jdbc(url=jdbcUrl, table='SalesLT.SalesOrderDetail', properties=connectionProperties, column='SalesOrderDetailID', lowerBound=1, upperBound=100000, numPartitions=100)
display(df)

SalesOrderID,SalesOrderDetailID,OrderQty,ProductID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
71774,110562,1,836,356.898,0.0,356.898,E3A1994C-7A68-4CE8-96A3-77FDD3BBD730,2008-06-01T00:00:00.000+0000
71774,110563,1,822,356.898,0.0,356.898,5C77F557-FDB6-43BA-90B9-9A7AEC55CA32,2008-06-01T00:00:00.000+0000
71776,110567,1,907,63.9,0.0,63.9,6DBFE398-D15D-425E-AA58-88178FE360E5,2008-06-01T00:00:00.000+0000
71780,110616,4,905,218.454,0.0,873.816,377246C9-4483-48ED-A5B9-E56F005364E0,2008-06-01T00:00:00.000+0000
71780,110617,2,983,461.694,0.0,923.388,43A54BCD-536D-4A1B-8E69-24D083507A14,2008-06-01T00:00:00.000+0000
71780,110618,6,988,112.998,0.4,406.7928,12706FAB-F3A2-48C6-B7C7-1CCDE4081F18,2008-06-01T00:00:00.000+0000
71780,110619,2,748,818.7,0.0,1637.4,B12F0D3B-5B4E-4F1F-B2F0-F7CDE99DD826,2008-06-01T00:00:00.000+0000
71780,110620,1,990,323.994,0.0,323.994,F117A449-039D-44B8-A4B2-B12001DACC01,2008-06-01T00:00:00.000+0000
71780,110621,1,926,149.874,0.0,149.874,92E5052B-72D0-4C91-9A8C-42591803667E,2008-06-01T00:00:00.000+0000
71780,110622,1,743,809.76,0.0,809.76,8BD33BED-C4F6-4D44-84FB-A7D04AFCD794,2008-06-01T00:00:00.000+0000


## Challenge

Write new records to the `SalesLT.ProductModel` Azure SQL Database table.

The first step is to return a count of records in the `SalesLT.ProductModel` table. We've provided the code to do this as a pushdown query (where you push down an entire query to the database and return just the result) below.

In [27]:
productmodel_count_query = "(select COUNT(*) count from SalesLT.ProductModel) count"
productmodel_count = spark.read.jdbc(url=jdbcUrl, table=productmodel_count_query, properties=connectionProperties)
display(productmodel_count)

count
128


In [28]:
product_model = spark.read.jdbc(url=jdbcUrl, table="SalesLT.ProductModel", properties=connectionProperties)

The second step is to define a schema that matches the data types in SQL. Since the primary key is an automatically generated identity column, you must exclude it from your schema since you will not be setting the value.

**Note:** You can get the schema with data types by retrieving the first row into a new DataFrame and printing the schema:

```python
productmodel_df = spark.read.jdbc(url=jdbcUrl, table="SalesLT.ProductModel", properties=connectionProperties).limit(1)
productmodel_df.printSchema
```

In [30]:
productmodel_df = spark.read.jdbc(url=jdbcUrl, table="SalesLT.ProductModel", properties=connectionProperties).limit(1)
productmodel_df.printSchema

Out[ 21 ]: <bound method DataFrame.printSchema of DataFrame[ProductModelID: int, Name: string, CatalogDescription: string, rowguid: string, ModifiedDate: timestamp]>

In [31]:
from pyspark.sql.types import *

schema = StructType([
  StructField("Name", StringType(), True),
  StructField("CatalogDescription", StringType(), True),
  StructField("rowguid", StringType(), True),
  StructField("ModifiedDate", TimestampType(), True)
])

Now, write new rows to the table, applying the schema from the `productModel_df` DataFrame.

In [33]:
# Import libraries to generate UUIDs (GUIDs), get the current date/time, set the save mode, and to use the Row object
import uuid
import datetime
from pyspark.sql import Row
newRows = [
  Row("Biking Shorts","They make you ride faster and look awesome doing it!", str(uuid.uuid4()), datetime.datetime.now()),
  Row("Biking Shirt","Glide through the atmosphere like Phoenix rising from the ashes", str(uuid.uuid4()), datetime.datetime.now())
]
parallelizeRows = spark.sparkContext.parallelize(newRows)
new_df = spark.createDataFrame(parallelizeRows, schema)

# TODO
new_df.write.jdbc(url=jdbcUrl, table="SalesLT.ProductModel", mode="append", properties=connectionProperties)
# Add code here to write to the SalesLT.ProductModel table

Do a final count on the `SalesLT.ProductModel` table to make sure the two new rows were inserted. You can also use the Query Editor in the Azure SQL Database portal interface to view your new rows there.

In [35]:
# TODO
productmodel_count = spark.read.jdbc(url=jdbcUrl, table=productmodel_count_query, properties=connectionProperties)
display(productmodel_count)
# Get a count of rows once more. The value should be 130

count
130


In [36]:
display(spark.read.jdbc(url=jdbcUrl, table="SalesLT.ProductModel", properties=connectionProperties))

ProductModelID,Name,CatalogDescription,rowguid,ModifiedDate
1,Classic Vest,null,29321D47-1E4C-4AAC-887C-19634328C25E,2007-06-01T00:00:00.000+0000
2,Cycling Cap,null,474FB654-3C96-4CB9-82DF-2152EEFFBDB0,2005-06-01T00:00:00.000+0000
3,Full-Finger Gloves,null,A75483FE-3C47-4AA4-93CF-664B51192987,2006-06-01T00:00:00.000+0000
4,Half-Finger Gloves,null,14B56F2A-D4AA-40A4-B9A2-984F165ED702,2006-06-01T00:00:00.000+0000
5,HL Mountain Frame,null,FDD5407B-C2DB-49D1-A86B-C13A2E3582A2,2005-06-01T00:00:00.000+0000
6,HL Road Frame,null,4D332ECC-48B3-4E04-B7E7-227F3AC2A7EC,2002-05-02T00:00:00.000+0000
7,HL Touring Frame,null,D60ED2A5-C100-4C54-89A1-531404C4A20F,2009-05-16T16:34:28.980+0000
8,LL Mountain Frame,null,65BF3F6D-BCF2-4DB6-8515-FC5C57423037,2006-11-20T09:56:38.273+0000
9,LL Road Frame,null,DDC67A2F-024A-4446-9B54-3C679BABA708,2005-06-01T00:00:00.000+0000
10,LL Touring Frame,null,66C63844-2A24-473C-96D5-D3B3FD57D834,2009-05-16T16:34:28.980+0000


## Next Steps

Start the next lesson, [Cosmos DB - Connect using Key Vault]($./10-Cosmos-DB-Connect-Using-Key-Vault)